In [2]:
import os
import json
import numpy as np
import pandas as pd

In [8]:
def calc_metric(metric_list, percent=100, std_flag=False):
    mean_val = np.mean(metric_list)
    std_val = np.std(metric_list)
    if std_flag:
        return f"{mean_val*percent:.2f}±{std_val:.2f}"
    else:
        return np.round(mean_val*percent, 2)
    

def summarize_json_to_df(json_folder, std_flag=False):
    metrics_list = []

    for filename in os.listdir(json_folder):
        if filename.endswith('.json'):
            with open(os.path.join(json_folder, filename), 'r') as file:
                metrics = json.load(file)
                edit_method, data, model = filename.split('_')[:3]

                if edit_method == 'IKE':
                    accuracy_pre = calc_metric([e['pre']['rewrite_acc'] for e in metrics])
                    accuracy_post = calc_metric([e['post']['rewrite_acc'] for e in metrics])
                else:
                    accuracy_pre = calc_metric([e['pre']['rewrite_acc'][0] for e in metrics])
                    accuracy_post = calc_metric([e['post']['rewrite_acc'][0] for e in metrics])

                results = {
                    "data": data,
                    "edit_method": edit_method,
                    "accuracy_pre": accuracy_pre,
                    "accuracy_post": accuracy_post,
                }
                metrics_list.append(results)
    df = pd.DataFrame(metrics_list)
    df = df[df['edit_method'] != 'SERAC']
    df = df[df['data'].isin(['recent', 'zsre', 'wikibio'])]
    return df


summarize_json_to_df("../results/know_edit/eval").sort_values(by=['data', 'edit_method'])

,data,edit_method,accuracy_pre,accuracy_post
18,recent,FT,47.40,100.00
0,recent,FT-L,47.40,56.30
17,recent,IKE,47.40,99.97
4,recent,LoRA,47.40,100.00
19,recent,MEMIT,47.40,97.10
12,recent,ROME,47.45,97.37
11,wikibio,FT,61.35,100.00
26,wikibio,FT-L,61.33,66.70
3,wikibio,IKE,61.33,100.00
13,wikibio,LoRA,61.33,100.00
